In [94]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

In [116]:
# import csv 

nba_data = pd.read_csv("nba_data.csv")
nba_data.count()

player_id           2625
Player              2625
Season              2625
G                   2625
GS                  2625
MP                  2625
FG                  2625
FGA                 2625
fgpercentage        2625
threePM             2625
threePA             2625
threePPercentage    2625
twoPM               2625
twoPA               2625
twoPPercentage      2625
efgpercentage       2625
FT                  2625
FTA                 2625
ftpercentage        2625
ORB                 2625
DRB                 2625
TRB                 2625
AST                 2625
STL                 2625
BLK                 2625
TOV                 2625
PF                  2625
PTS                 2625
PER                 2625
BPM                 2625
VORP                2625
dtype: int64

In [117]:
# Remove extreme outlier
nba_data['G'] = nba_data['G'].astype(float)
#Using drop() to delete rows based on column value
nba_data = nba_data.drop(nba_data[nba_data['G'] <= 6].index) 
nba_data = nba_data.drop(nba_data[nba_data['MP'] <= 3].index)
nba_data.count()

player_id           2393
Player              2393
Season              2393
G                   2393
GS                  2393
MP                  2393
FG                  2393
FGA                 2393
fgpercentage        2393
threePM             2393
threePA             2393
threePPercentage    2393
twoPM               2393
twoPA               2393
twoPPercentage      2393
efgpercentage       2393
FT                  2393
FTA                 2393
ftpercentage        2393
ORB                 2393
DRB                 2393
TRB                 2393
AST                 2393
STL                 2393
BLK                 2393
TOV                 2393
PF                  2393
PTS                 2393
PER                 2393
BPM                 2393
VORP                2393
dtype: int64

In [118]:
# create advanced columns df for the machine learning model

advanced = [nba_data["G"],nba_data["MP"], nba_data["PER"],nba_data["BPM"],nba_data["VORP"]]
advanced_df = pd.concat(advanced, axis=1)
advanced_df

,G,MP,PER,BPM,VORP
0,68.0,15.5,10.1,-1.6,0.1
1,38.0,14.7,11.8,-2.1,0.0
2,80.0,29.9,16.5,-0.2,1.1
3,61.0,25.9,8.9,-3.6,-0.7
4,39.0,15.0,12.9,-3.3,-0.2
...,...,...,...,...,...
2620,63.0,27.7,16.3,2.2,1.8
2621,68.0,24.3,20.3,3.3,2.2
2622,63.0,33.7,23.0,3.7,3.0
2623,48.0,20.9,18.2,-0.5,0.4


In [119]:
# scale the advanced df

nba_scaled = StandardScaler().fit_transform(advanced_df)
print(nba_scaled[0:5])

[[ 0.76476195 -0.58127607 -0.66293725 -0.1377741  -0.39371766]
 [-0.61545477 -0.67283399 -0.32734393 -0.29229857 -0.47298456]
 [ 1.31684864  1.06676642  0.60047289  0.29489442  0.39895133]
 [ 0.44271139  0.60897684 -0.89982665 -0.75587198 -1.02785284]
 [-0.56944754 -0.63849977 -0.11019531 -0.6631573  -0.63151835]]


In [120]:
# Initialize PCA model
pca = PCA(n_components=3)

In [121]:
# Get 3 principal components for the data.
nba_pca = pca.fit_transform(nba_scaled)

In [122]:
# create PCA df
df_nba_pca = pd.DataFrame(
    data = nba_pca, columns = ["principal component 1", "principal component 2", "principal component 3"], index=nba_data.index)
df_nba_pca.head()

,principal component 1,principal component 2,principal component 3
0,-0.549030,-0.691784,-0.762504
1,-1.025293,0.415434,-0.103336
2,1.531965,-1.005637,-0.089367
3,-0.877664,-1.394199,0.340511
4,-1.153116,0.310684,-0.114554


In [123]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_nba_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [125]:
# make "Player" and "Year" its own df

player = [nba_data["Player"],nba_data["Season"]]
player_name = pd.concat(player, axis=1)
player_name.head()

,Player,Season
0,Álex Abrines,2016-17
1,Quincy Acy,2016-17
2,Steven Adams,2016-17
3,Arron Afflalo,2016-17
4,Alexis Ajinça,2016-17


In [126]:
# Initialize the K-means model
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(df_nba_pca)

# Predict clusters
predictions = model.predict(df_nba_pca)

# Add the predicted class columns
player_name["class"] = model.labels_
player_name.head()

,Player,Season,class
0,Álex Abrines,2016-17,0
1,Quincy Acy,2016-17,1
2,Steven Adams,2016-17,3
3,Arron Afflalo,2016-17,0
4,Alexis Ajinça,2016-17,1


In [127]:
#sort by class

sorted_class = player_name.sort_values(["class"])
class_one = sorted_class[(sorted_class["class"] == 0)]
class_two = sorted_class[(sorted_class["class"] == 1)]
class_three = sorted_class[(sorted_class["class"] == 2)]
class_four = sorted_class[(sorted_class["class"] == 3)]
class_five = sorted_class[(sorted_class["class"] == 4)]

In [146]:
# class 1 df

df_one = pd.concat([class_one, advanced_df], axis=1)
class_one_df = df_one.dropna()
class_one_df.sample(60)

,Player,Season,class,G,MP,PER,BPM,VORP
1600,Bismack Biyombo,2019-20,0.0,53.0,19.4,16.1,-2.5,-0.1
1749,Rui Hachimura,2019-20,0.0,48.0,30.1,13.5,-3.2,-0.4
1143,Torrey Craig,2018-19,0.0,75.0,20.0,10.4,-1.9,0.0
1274,Brandon Ingram,2018-19,0.0,52.0,33.8,13.4,-2.0,0.0
720,Andre Iguodala,2017-18,0.0,64.0,25.3,11.2,-0.3,0.7
82,Jordan Clarkson,2016-17,0.0,82.0,29.2,13.2,-2.5,-0.3
1666,Pat Connaughton,2019-20,0.0,67.0,18.6,11.5,0.6,0.8
1332,Shaun Livingston,2018-19,0.0,64.0,15.1,11.7,-1.0,0.2
393,Tomáš Satoranský,2016-17,0.0,57.0,12.6,8.5,-2.4,-0.1
1729,Taj Gibson,2019-20,0.0,62.0,16.5,16.1,-1.1,0.2


In [134]:
# class 2 df

df_two = pd.concat([class_two, advanced_df], axis=1)
class_two_df = df_two.dropna()
class_two_df.tail()

,Player,Season,class,G,MP,PER,BPM,VORP
2599,Paul Watson,2020-21,1.0,27.0,11.0,10.9,-1.3,0.1
2604,Hassan Whiteside,2020-21,1.0,36.0,15.2,19.2,-2.2,0.0
2613,D.J. Wilson,2020-21,1.0,35.0,12.4,11.5,-3.3,-0.1
2614,Dylan Windler,2020-21,1.0,31.0,16.5,10.8,-1.7,0.0
2623,Cody Zeller,2020-21,1.0,48.0,20.9,18.2,-0.5,0.4


In [135]:
# class 3 df

df_three = pd.concat([class_three, advanced_df], axis=1)
class_three_df = df_three.dropna()
class_three_df.tail()

,Player,Season,class,G,MP,PER,BPM,VORP
2586,Gabe Vincent,2020-21,2.0,50.0,13.1,7.6,-5.4,-0.6
2600,Quinndary Weatherspoon,2020-21,2.0,20.0,6.1,9.6,-5.0,-0.1
2615,Justise Winslow,2020-21,2.0,26.0,19.5,6.2,-6.0,-0.5
2616,Cassius Winston,2020-21,2.0,22.0,4.5,10.8,-3.2,0.0
2619,Robert Woodard II,2020-21,2.0,13.0,3.5,12.5,-5.3,0.0


In [141]:
# class 4 df

df_four = pd.concat([class_four, advanced_df], axis=1)
class_four_df = df_four.dropna()
class_four_df.tail(50)

,Player,Season,class,G,MP,PER,BPM,VORP
2413,T.J. McConnell,2020-21,3.0,69.0,26.0,16.9,1.4,1.5
2416,Doug McDermott,2020-21,3.0,66.0,24.5,15.6,-0.6,0.6
2424,De'Anthony Melton,2020-21,3.0,52.0,20.1,15.2,2.3,1.1
2427,Khris Middleton,2020-21,3.0,68.0,33.4,18.2,1.3,1.9
2430,Paul Millsap,2020-21,3.0,56.0,20.8,16.4,1.0,0.9
2436,Ja Morant,2020-21,3.0,63.0,32.6,16.7,-0.4,0.8
2438,Marcus Morris,2020-21,3.0,57.0,26.4,14.5,0.0,0.8
2440,Monte Morris,2020-21,3.0,47.0,25.4,14.0,-0.3,0.5
2442,Dejounte Murray,2020-21,3.0,67.0,31.9,16.5,0.8,1.5
2443,Jamal Murray,2020-21,3.0,48.0,35.5,18.3,2.0,1.7


In [142]:
# class 5 df

df_five = pd.concat([class_five, advanced_df], axis=1)
class_five_df = df_five.dropna()
class_five_df.tail(50)

,Player,Season,class,G,MP,PER,BPM,VORP
1852,Damian Lillard,2019-20,4.0,66.0,37.5,26.9,7.5,5.9
1858,Kyle Lowry,2019-20,4.0,58.0,36.2,17.7,3.2,2.8
1891,Khris Middleton,2019-20,4.0,62.0,29.9,21.1,4.1,2.9
1951,Chris Paul,2019-20,4.0,70.0,31.5,21.7,4.4,3.5
1991,Domantas Sabonis,2019-20,4.0,62.0,34.8,20.7,3.2,2.9
2006,Ben Simmons,2019-20,4.0,57.0,35.4,20.4,3.6,2.8
2019,Jayson Tatum,2019-20,4.0,66.0,34.3,20.4,4.0,3.4
2032,Karl-Anthony Towns,2019-20,4.0,35.0,33.9,26.5,7.8,2.9
2046,Nikola Vučević,2019-20,4.0,62.0,32.2,21.9,4.2,3.1
2050,Kemba Walker,2019-20,4.0,56.0,31.1,20.0,4.6,2.9
